In [1]:
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/63714679/plotting-gannt-chart-using-timestamps
# https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587

# ******** open this
# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4

# https://plotly.com/python/gantt/
# https://plotly.com/python-api-reference/generated/plotly.express.timeline.html
# https://plotly.com/python/reference/layout/yaxis/

# for when you want to reduce the number of dimensions of a plot, 
# i.e. make another chart with the dimension separated out, i.e. multiple charts of different y-variables
# https://plotly.com/python/facet-plots/
# *******

import numpy as np
import pandas as pd
import plotly.express as px
import json
import os

import jupyter_dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime as dt, date, time, timedelta

from functools import reduce
from scipy.stats import pearsonr

from plotly_calplot import calplot

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [9]:
#report = '~/qs/ml/data/time_2020-present.csv'
#report = '~/qs/ml/data/time_2020-2021.csv'
#report = '~/qs/ml/data/time_2021-2022.csv'
#report = '~/qs/ml/data/time_2022-present.csv'
report = '~/qs/ml/data/time_2020-present.csv'

df = pd.read_csv(report)
df

,Group,Activity type,Duration,From,To,Comment
0,NaN,Toilet,0:17:55,2023-03-12 23:56:28,2023-03-13 0:14:23,NaN
1,NaN,Studying,0:19:21,2023-03-12 23:35:58,2023-03-12 23:55:19,NaN
2,NaN,Eating,0:11:26,2023-03-12 23:14:19,2023-03-12 23:25:45,NaN
3,NaN,Toilet,0:02:06,2023-03-12 22:38:49,2023-03-12 22:40:55,NaN
4,NaN,Studying,1:00:01,2023-03-12 21:35:43,2023-03-12 22:35:44,NaN
...,...,...,...,...,...,...
30157,NaN,Toilet,0:01:35,2019-12-31 1:27:22,2019-12-31 1:28:57,NaN
30158,NaN,Grooming,0:05:42,2019-12-31 1:21:40,2019-12-31 1:27:22,NaN
30159,Computer,Internet,0:08:01,2019-12-31 1:13:03,2019-12-31 1:21:04,NaN
30160,Personal,Masturbation,0:06:19,2019-12-31 1:06:36,2019-12-31 1:12:55,NaN


In [10]:
def transform_df(df=df):
        original_df = df = df[['Group', 'Activity type', 'From', 'To']]
        original_df = df = df.rename(columns={'Activity type':'Activity', 
                                              'From':'Start', 
                                              'To':'End'})
        df['Day'] = original_df['Start'].str[0:10]
        original_df['Start'] = df['Start'] = pd.to_datetime(df['Start'])
        original_df['End'] = df['End'] = pd.to_datetime(df['End'])
        replace_activity_name(df)
        replace_activity_name(original_df)
        
        original_df['Duration in Seconds'] = df['Duration in Seconds'] = pd.to_timedelta(df['End'] - df['Start']).astype('timedelta64[s]')
        original_df['Duration'] = pd.to_datetime(df["Duration in Seconds"], unit='s').dt.strftime("%H:%M:%S")
        
        edges = pd.date_range(df.Start.min().normalize() - pd.Timedelta(days=1), 
                              df.End.max().normalize() + pd.Timedelta(days=1), freq='D')
        l = []
        for edge in edges:
            m = df.Start.lt(edge) & df.End.gt(edge)            # Rows to modify
            l.append(df.loc[m].assign(End=edge))               # Clip end of modified rows
            df.loc[m, 'Start'] = edge                          # Fix start for next edge
            
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
        df = pd.concat(l+[df], ignore_index=True).sort_values('Start')
        
        df['Day'] = df['Start'].dt.date
        df['Start'] = pd.to_datetime(df.Start)
        df["Start"] = df["Start"].apply(lambda x: x.replace(year=1970, month=1, day=1))
        df['Start'] = pd.to_datetime(df.Start)

        df['End'] = pd.to_datetime(df.End)
        df["End"] = df["End"].apply(lambda x: x.replace(year=1970, month=1, day=1))
        df['End'] = pd.to_datetime(df.End)
        
        m = dt(year=1970, month=1, day=1, hour=0, minute=0, second=0)
        new = dt(year=1970, month=1, day=1, hour=0, minute=0, second=1)
        df['Start'] = df['Start'].mask(df['Start'] == m, new)
        m = dt(year=1970, month=1, day=1, hour=0, minute=0, second=0)
        new = dt(year=1970, month=1, day=1, hour=23, minute=59, second=59)
        df['End'] = df['End'].mask(df['End'] == m, new)
        df['Duration in Seconds'] = pd.to_timedelta(df['End'] - df['Start']).astype('timedelta64[s]')
        df['Duration'] = pd.to_datetime(df["Duration in Seconds"], unit='s').dt.strftime("%H:%M:%S")
        
        df.to_csv('data/split_activities.csv', index=None, encoding='utf-8')
        original_df.to_csv('data/activities.csv', index=None, encoding='utf-8')
        return original_df, df
    
def replace_activity_name(df=df):
        for activity in df['Activity'].unique():
            print(activity)
        print()

        while True:
            replace = input("Do you want to replace any activity's names with something else? (y/n): \n")
            try:
                if replace == 'y' or replace == 'Y' or replace == 'yes' or replace == 'Yes' :
                    replacement_activity = input('What activity do you want to replace?: ')
                    replaced_with = input('And what do you want to replace it with?: ')
                    df.loc[df['Activity'].str.contains(replacement_activity), 'Activity'] = replaced_with
                    original_df.loc[df['Activity'].str.contains(replacement_activity), 'Activity'] = replaced_with
                    print(f'Replaced: {replacement_activity} with {replaced_with}')
                    continue
                elif replace == "n" or replace == "N" or replace == "no" or replace == "No" or replace == "" or replace == " ":
                    print("Finished replacing activity names.")
                else:
                    raise Exception("Answer must be some variation of 'yes' or 'no.'")

            except Exception as e:
                print(e)

            else:
                for activity in df['Activity'].unique():
                    print(activity)
                break

In [11]:
original_df, df = transform_df(df)

Toilet
Studying
Eating
Reading
Working
Walking
Grooming
Sleeping
Masturbation
Internet
Cooking
Driving
Gym
Cleaning
Family Time
Class
Drinking
Friends Time
Phone
Shopping
Dishes
GameDev
Errands and Appointments
Movies/TV
Journaling
Programming
Writing
Games
Dating
Drumming
Traveling 
Club/Social Group
Mycology
Swimming/Tanning
Concert/Event
Biking
Dancing
Sex
Drawing
Data Analysis
Keyboard



Do you want to replace any activity's names with something else? (y/n): 
 y
What activity do you want to replace?:  Masturbation
And what do you want to replace it with?:  Personal


Replaced: Masturbation with Personal


Do you want to replace any activity's names with something else? (y/n): 
 y
What activity do you want to replace?:  Sex
And what do you want to replace it with?:  Personal


Replaced: Sex with Personal


Do you want to replace any activity's names with something else? (y/n): 
 n


Finished replacing activity names.
Toilet
Studying
Eating
Reading
Working
Walking
Grooming
Sleeping
Personal
Internet
Cooking
Driving
Gym
Cleaning
Family Time
Class
Drinking
Friends Time
Phone
Shopping
Dishes
GameDev
Errands and Appointments
Movies/TV
Journaling
Programming
Writing
Games
Dating
Drumming
Traveling 
Club/Social Group
Mycology
Swimming/Tanning
Concert/Event
Biking
Dancing
Drawing
Data Analysis
Keyboard
Toilet
Studying
Eating
Reading
Working
Walking
Grooming
Sleeping
Personal
Internet
Cooking
Driving
Gym
Cleaning
Family Time
Class
Drinking
Friends Time
Phone
Shopping
Dishes
GameDev
Errands and Appointments
Movies/TV
Journaling
Programming
Writing
Games
Dating
Drumming
Traveling 
Club/Social Group
Mycology
Swimming/Tanning
Concert/Event
Biking
Dancing
Drawing
Data Analysis
Keyboard



Do you want to replace any activity's names with something else? (y/n): 
 n


Finished replacing activity names.
Toilet
Studying
Eating
Reading
Working
Walking
Grooming
Sleeping
Personal
Internet
Cooking
Driving
Gym
Cleaning
Family Time
Class
Drinking
Friends Time
Phone
Shopping
Dishes
GameDev
Errands and Appointments
Movies/TV
Journaling
Programming
Writing
Games
Dating
Drumming
Traveling 
Club/Social Group
Mycology
Swimming/Tanning
Concert/Event
Biking
Dancing
Drawing
Data Analysis
Keyboard


In [12]:
# # make easy to paste activity variables
# for activity in df['Activity type'].unique():
#     print(f"{activity.lower().replace('/', '_').replace(' ', '_')} = ")

In [13]:
working = '#D50000'              #custom_colors[0][3]
sleeping = '#2196F3'             #custom_colors[6][0]
grooming = '#80D8FF'             #custom_colors[8][0]
internet = '#00C853'             #custom_colors[10][3]
movies_tv = '#00C853'
games = '#B9F6CA'
programming = '#00E676'
gamedev = '#00E676'

walking = '#BDBDBD'
shopping = '#32514B'
eating = '#18FFFF'
cooking = '#00BFA5'
cleaning = '#1DE9B6'
dishes = '#64FFDA'

driving = '#757575'
gym = '#9C27B0'
masturbation = '#FF80AB'         #custom_colors[7][1]
sex = '#FF80AB' 
personal = '#FF80AB' 
toilet = '#FFFF8D'               #custom_colors[13][1]

classes = '#BCA68E' 
drinking = '#00B8D4'
phone = '#D7CCC8'
drumming = '#8C9EEE'
writing = '#536DFE'
reading = '#3D5AFE'
keyboard = '#E040FB'
drawing = '#AA00FF'
swimming = '#80D8FF'
journaling = '#E040FB'
traveling = '#A1887F'
biking = '#651FFF'
mycology = '#EA80FC'

studying = '#BCA68E' 
errands = '#A1887F'
appointments = '#A1887F'
data_analysis = '#78909C'
dating = '#FF80AB'
family_time = '#FF9100'
friends_time = '#FFAB40'
event = '#AA00FF'
dancing = '#D500F9'
club = '#FFD180'

In [14]:
# https://stackoverflow.com/questions/63460213/plotly-how-to-define-colors-in-a-figure-using-plotly-graph-objects-and-plotly-e
# https://community.plotly.com/t/different-colors-for-bars-in-barchart-by-their-value/6527/7
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph

colors = {'Working': f'{working}',
          'Sleeping': f'{sleeping}',
          'Grooming': f'{grooming}',
          'Internet': f'{internet}',
          'Movies/TV': f'{movies_tv}',
          'Programming': f'{programming}',
          'Games': f'{games}',
          'GameDev': f'{gamedev}',
          
          'Walking': f'{walking}',
          'Shopping': f'{shopping}',
          'Eating': f'{eating}',
          'Cooking': f'{cooking}',
          'Cleaning': f'{cleaning}',
          'Dishes': f'{dishes}',
          
          'Driving': f'{driving}',
          'Gym': f'{gym}',
          'Masturbation': f'{masturbation}',
          'Sex': f'{sex}',
          'Personal': f'{personal}',
          'Toilet': f'{toilet}',
          
          'Class': f'{classes}',
          'Drinking': f'{drinking}',
          'Phone': f'{phone}',
          'Drumming': f'{drumming}',
          'Writing': f'{writing}',
          'Reading': f'{reading}',
          'Keyboard': f'{keyboard}',
          'Drawing': f'{drawing}',
          'Swimming/Tanning': f'{swimming}',
          'Journaling': f'{journaling}',
          'Traveling': f'{traveling}',
          'Biking': f'{biking}',
          'Mycology': f'{mycology}',
          
          'Studying': f'{studying}',
          'Errands': f'{errands}',
          'Appointments': f'{appointments}',
          'Errands and Appointments': f'{errands}',
          'Data Analysis': f'{data_analysis}',
          'Dating': f'{dating}',
          'Family Time': f'{family_time}',
          'Friends Time': f'{friends_time}',
          'Event': f'{event}',
          'Dancing': f'{dancing}',
          'Club/Social Group': f'{club}',
         }

colors

{'Working': '#D50000',
 'Sleeping': '#2196F3',
 'Grooming': '#80D8FF',
 'Internet': '#00C853',
 'Movies/TV': '#00C853',
 'Programming': '#00E676',
 'Games': '#B9F6CA',
 'GameDev': '#00E676',
 'Walking': '#BDBDBD',
 'Shopping': '#32514B',
 'Eating': '#18FFFF',
 'Cooking': '#00BFA5',
 'Cleaning': '#1DE9B6',
 'Dishes': '#64FFDA',
 'Driving': '#757575',
 'Gym': '#9C27B0',
 'Masturbation': '#FF80AB',
 'Sex': '#FF80AB',
 'Personal': '#FF80AB',
 'Toilet': '#FFFF8D',
 'Class': '#BCA68E',
 'Drinking': '#00B8D4',
 'Phone': '#D7CCC8',
 'Drumming': '#8C9EEE',
 'Writing': '#536DFE',
 'Reading': '#3D5AFE',
 'Keyboard': '#E040FB',
 'Drawing': '#AA00FF',
 'Swimming/Tanning': '#80D8FF',
 'Journaling': '#E040FB',
 'Traveling': '#A1887F',
 'Biking': '#651FFF',
 'Mycology': '#EA80FC',
 'Studying': '#BCA68E',
 'Errands': '#A1887F',
 'Appointments': '#A1887F',
 'Errands and Appointments': '#A1887F',
 'Data Analysis': '#78909C',
 'Dating': '#FF80AB',
 'Family Time': '#FF9100',
 'Friends Time': '#FFAB40',
 'E

In [15]:
# def make_colors(colors=colors, filepath='data/colors.json'):
#     with open(filepath, 'w') as json_file:
#         json.dump(colors, json_file)

In [16]:
# make_colors()

In [17]:
# https://www.codegrepper.com/code-examples/python/how+to+create+dynamic+variable+names+in+python

# make rolling averages, only will work if all the average_activity_df's have the same day
# make time since last instance
# https://stackoverflow.com/questions/51856844/python-pandas-minutes-since-last-occurrence-in-2-million-row-dataframe
# add longest streak of activities, i.e. 365 days in a row on internet or 7 days in a row gaming, etc.

# make .corr matrix of instances per day/durations per day and .cov (covariance)
# make data table
# make heatmap of per activity instance in day and of total amount of durations in day
# make histograms of each activity's durations per day vs instances at time those durations, bin = minute?
# make violin/ridgeplots for each of the activities as a y, https://plotly.com/python/violin/
# make normal distribution histogram, mean/median = middle, std = lines
# show "Created _df, sum_, mean_, median_, etc."

# gmean, hmean, zscores
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gmean.html#scipy.stats.gmean
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.hmean.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html
# https://www.simplypsychology.org/z-score.html
# range, absolute minimum vs calculated minimum, i.e. 0 seconds vs of values above 0
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.iqr.html 75th q - 25th q, gets rid of outliers

# make interpolated counts df, for each activity, then mean/median/sum/quantile/min/max df of all of the counts of activities

# make the splitting of the df's at midnight a part of this class __init__ to get accurate activity_df['Start']/['End']'s, without the 1970 though

class ActivityStats:
    def __init__(self, df=original_df, df2=df):
        self.df = df
        self.unique_activities = self.df['Activity'].unique().tolist()
        self.days_in_df = pd.DataFrame(self.df['Day'].unique(), columns=['Day'])
        days_to_remove = 1 # because the first day from the aTimeLogger .csv is not supposed to be counted, as it is not a full day, but includes any activities going into the next day at 24:00/0:00
        self.num_days_in_df = len(self.days_in_df) - days_to_remove
        seconds_in_day = 86400
        self.seconds_in_df = self.num_days_in_df * seconds_in_day
        
        globals()["unique_activities"] = self.unique_activities
        globals()["days_in_df"] = self.days_in_df
        globals()["num_days_in_df"] = self.num_days_in_df
        globals()["seconds_in_df"] = self.seconds_in_df
                
        for activity in self.unique_activities:
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'])]
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'])]
            
#             globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'].mean()))
#             globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'].median()))
#             globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'].mean()))
#             globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'].median()))
            
            # start of activity df's for heatmap from above, i.e. insert whatever you make above into here
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Day']
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Duration in Seconds'] =  df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']
            
            
            
            
            
            
# percent of activities of total activity per day, i.e., 3 instances of internet in day, 24 activities, 3/24 = 12.5% of activities in day was internet.
# percent of days that I engaged in that activity, i.e. I engaged in internet 365 days in year == 100%

            globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df2.loc[df2['Activity'].str.contains(f"{activity}"), 'Start'].mean()).round(freq='S').time()
            globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df2.loc[df2['Activity'].str.contains(f"{activity}"), 'Start'].median()).round(freq='S').time()
            globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df2.loc[df2['Activity'].str.contains(f"{activity}"), 'End'].mean()).round(freq='S').time()
            globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df2.loc[df2['Activity'].str.contains(f"{activity}"), 'End'].median()).round(freq='S').time()

            globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].sum()
            globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).mean()
            globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).median()
            globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = np.nan_to_num(round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).std())
            globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].min()
            globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].max()

            globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] = len(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')])
            globals()[f"mean_num_{activity}_per_day".lower().replace('/', '_').replace(' ', '_')] = round(globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] / self.num_days_in_df, 2)
            globals()[f"{activity}_percent".lower().replace('/', '_').replace(' ', '_')] = (globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] / self.seconds_in_df) * 100
    
            globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]))[-8:]
            globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]).days)
            globals()[f"sum_{activity}_dhms".lower().replace('/', '_').replace(' ', '_')] = globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] + ':' + globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')]
            globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            
            globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).quantile(.25)
            globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).quantile(.75)
            globals()[f"25_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"75_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            
        total_seconds_list = []
        for activity in unique_activities:
            total_seconds_list.append([f"{activity}", globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]])
        total_seconds_list = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
        unique_activities_sorted = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
        self.unique_activities_sorted = [i[0] for i in unique_activities_sorted]
        globals()["unique_activities_sorted"] = self.unique_activities_sorted
   
    def activities(self):
        return [activity for activity in self.unique_activities]

    def min_durations(self):
        globals()['min_activity_durations_hms'] = pd.DataFrame()
        globals()['min_activity_durations_hms']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['min_activity_durations_hms']['min_activity_durations_hms'] = [globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        print("Created 'min_activity_durations_hms'")
        return globals()['min_activity_durations_hms']

    def max_durations(self):
        globals()['max_activity_durations_hms'] = pd.DataFrame()
        globals()['max_activity_durations_hms']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['max_activity_durations_hms']['max_activity_durations_hms'] = [globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        print("Created 'max_activity_durations_hms'")
        return globals()['max_activity_durations_hms']     

    def mean_std_durations(self):
        globals()['mean_std_durations'] = pd.DataFrame()
        globals()['mean_std_durations']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['mean_std_durations']['mean_std_durations'] = [globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] + 
                                                                 ' \u00B1 ' + 
                                                                 globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')]
                                                                 for activity in self.unique_activities_sorted]
        print("Created 'mean_std_durations'")
        return globals()['mean_std_durations']

    # make median_std_durations like above
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_abs_deviation.html#scipy.stats.median_abs_deviation
    #https://stackoverflow.com/questions/61436389/use-custom-function-median-absolute-deviation-in-pandas-transform
    #https://stackoverflow.com/questions/22804036/median-absolute-deviation-mad-based-outlier-detection
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_absolute_deviation.html
    
    # make percentile_durations with required argument of int(0-100)
    # np.percentile, pd.percentile
    
    def describe(self, ):
        
        # activity_df
        globals()['activity_df'] = pd.DataFrame()
        globals()['activity_df_seconds'] = pd.DataFrame()
#         globals()['activity_df_times'] = pd.DataFrame()

        globals()['activity_df']['Activity'] = [activity for activity in self.unique_activities_sorted]
        globals()['activity_df']['total_num_instance_activity'] = [globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_num_activity_per_day'] = [globals()[f"mean_num_{activity}_per_day".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['activity_percent'] = [globals()[f"{activity}_percent".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df']['mean_activity_start'] = [globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_start'] = [globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_activity_end'] = [globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_end'] = [globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
# rolling averages instances etc
        
        globals()['activity_df_seconds']['Activity'] = [activity for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['sum_activity_durations_seconds'] = [globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['mean_activity_durations_seconds'] = [globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['std_activity_durations_seconds'] = [globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['min_activity_durations_seconds'] = [globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['25_quant_activity_durations'] = [globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['median_activity_durations_seconds'] = [globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['75_quant_activity_durations'] = [globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['max_activity_durations_seconds'] = [globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

#         globals()['activity_df']['sum_activity_timedelta'] = [globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
#         globals()['activity_df']['sum_activity_daydelta'] = [globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df']['sum_activity_dhms'] = [globals()[f"sum_{activity}_dhms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_activity_durations_hms'] = [globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['std_activity_durations_hms'] = [globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['min_activity_durations_hms'] = [globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]       
        globals()['activity_df']['25_quant_activity_durations_hms'] = [globals()[f"25_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_durations_hms'] = [globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['75_quant_activity_durations_hms'] = [globals()[f"75_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['max_activity_durations_hms'] = [globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df_transposed'] = globals()['activity_df'].set_index('Activity').T
        globals()['activity_df_transposed'] = globals()['activity_df_transposed'].reset_index()
        globals()['activity_df_seconds_transposed'] = globals()['activity_df_seconds'].set_index('Activity').T
        globals()['activity_df_seconds_transposed'] = globals()['activity_df_seconds_transposed'].reset_index()

        # activity_df_totals
        globals()['activity_df_totals'] = pd.DataFrame()
        globals()['activity_df_totals']['total_durations_seconds_in_df'] = [globals()['activity_df_seconds']['sum_activity_durations_seconds'].sum()]        
        globals()[f"total_durations_timedelta"] = str(pd.Timedelta(seconds=globals()['activity_df_totals']['total_durations_seconds_in_df'][0]))[-8:]
        globals()[f"total_durations_daydelta"] = str(pd.Timedelta(seconds=globals()['activity_df_totals']['total_durations_seconds_in_df'][0]).days)
        globals()['activity_df_totals']['total_durations_dhms_in_df'] = globals()[f"total_durations_daydelta"] + ':' + globals()[f"total_durations_timedelta"]
        globals()['activity_df_totals']['total_days_in_df'] = self.num_days_in_df
        globals()['activity_df_totals']['num_unique_activities_in_df'] = len(self.unique_activities)
        globals()['activity_df_totals']['total_instances_of_activities_in_df'] = [globals()['activity_df']['total_num_instance_activity'].sum()]
        
        self.total_activities_per_day_df = pd.DataFrame(df.groupby(df['Day']).agg('Duration').count().reset_index()).rename({'Duration':'Counts'}, axis='columns')
        globals()["total_activities_per_day_df"] = self.total_activities_per_day_df
        
        self.total_activities_per_day_mean = round(self.total_activities_per_day_df['Counts'].mean(), 2)
        self.total_activities_per_day_std = round(self.total_activities_per_day_df['Counts'].std(), 2)
        self.total_activities_per_day_min = int(self.total_activities_per_day_df['Counts'].min())
        self.total_activities_per_day_25 = int(self.total_activities_per_day_df['Counts'].quantile(.25))
        self.total_activities_per_day_median = int(self.total_activities_per_day_df['Counts'].median())
        self.total_activities_per_day_75 = int(self.total_activities_per_day_df['Counts'].quantile(.75))
        self.total_activities_per_day_max = int(self.total_activities_per_day_df['Counts'].max())
        
        globals()['activity_df_totals']['total_activities_per_day_mean'] = [self.total_activities_per_day_mean]
        globals()['activity_df_totals']['total_activities_per_day_std'] = [self.total_activities_per_day_std]
        globals()['activity_df_totals']['total_activities_per_day_min'] = [self.total_activities_per_day_min]
        globals()['activity_df_totals']['total_activities_per_day_25'] = [self.total_activities_per_day_25]
        globals()['activity_df_totals']['total_activities_per_day_median'] = [self.total_activities_per_day_median]
        globals()['activity_df_totals']['total_activities_per_day_75'] = [self.total_activities_per_day_75]
        globals()['activity_df_totals']['total_activities_per_day_max'] = [self.total_activities_per_day_max]
        
#         globals()['total_activities_per_day_mean'] = self.activities_per_day_mean
#         globals()['total_activities_per_day_std'] = self.activities_per_day_std
#         globals()['total_activities_per_day_25'] = self.activities_per_day_25
#         globals()['total_activities_per_day_median'] = self.activities_per_day_median
#         globals()['total_activities_per_day_75'] = self.activities_per_day_75
#         globals()['total_activities_per_day_max'] = self.activities_per_day_max

        
        globals()['activity_df_totals'] = globals()['activity_df_totals'].T
        globals()['activity_df_totals'] = globals()['activity_df_totals'].rename(columns={0:'activity_df_totals'})
        globals()['activity_df_totals'] = globals()['activity_df_totals'].reset_index()
        #globals()['activity_df_totals'] = globals()['activity_df_totals'].set_index('index')
        
        print("Created 'activity_df,' 'activity_df_transposed,'")
        print("Created 'activity_df_seconds', 'activity_df_seconds_transposed,'")
        print("Created 'activity_df_totals,'")
        return globals()['activity_df_transposed']

In [18]:
ActivityStats(df=original_df, df2=df).describe()

AttributeError: 'list' object has no attribute 'groupby'

Created 'activity_df,' 'activity_df_transposed,'
Created 'activity_df_seconds', 'activity_df_seconds_transposed,'
Created 'activity_df_totals,'


Activity,index,Sleeping,Internet,Working,Games,Family Time,Eating,Programming,Grooming,Reading,Gym,Toilet,Friends Time,Studying,Movies/TV,Driving,Walking,Writing,Cooking,Class,Cleaning,Dishes,Personal,Dating,Shopping,Mycology,Errands and Appointments,Club/Social Group,Drinking,GameDev,Swimming/Tanning,Phone,Keyboard,Journaling,Drawing,Traveling,Drumming,Data Analysis,Concert/Event,Biking,Dancing
0,total_num_instance_activity,1351,4312,551,355,219,3533,736,2364,1415,386,8238,64,359,206,1564,1402,274,1051,107,326,136,688,29,133,57,88,11,52,9,15,45,6,45,3,2,16,10,2,1,1
1,mean_num_activity_per_day,1.16,3.69,0.47,0.3,0.19,3.02,0.63,2.02,1.21,0.33,7.05,0.05,0.31,0.18,1.34,1.2,0.23,0.9,0.09,0.28,0.12,0.59,0.02,0.11,0.05,0.08,0.01,0.04,0.01,0.01,0.04,0.01,0.04,0.0,0.0,0.01,0.01,0.0,0.0,0.0
2,activity_percent,34.215101,23.1822,22.994834,3.755453,2.935969,2.859707,2.800872,2.23948,1.958512,1.496548,1.430974,1.288366,1.286875,1.226583,0.98966,0.971356,0.701818,0.58245,0.536945,0.410868,0.386413,0.356662,0.323522,0.199049,0.111682,0.102975,0.085917,0.082043,0.06525,0.062019,0.054498,0.046651,0.038294,0.02442,0.024051,0.023542,0.020955,0.01484,0.006591,0.000939
3,mean_activity_start,09:33:52,12:10:07,09:58:04,09:06:27,15:24:50,12:14:13,06:29:55,14:00:12,06:22:43,09:53:45,12:13:13,12:23:03,07:50:48,10:45:58,13:29:15,13:18:55,07:26:03,12:16:42,10:39:05,16:01:48,13:02:11,11:15:55,14:09:16,17:14:18,14:43:26,12:52:55,14:04:51,19:03:04,13:14:27,14:50:17,12:16:43,14:38:34,18:31:52,09:33:29,07:37:20,18:16:28,05:36:05,15:49:32,13:57:33,23:24:14
4,median_activity_start,10:41:17,11:31:13,07:56:00,06:45:48,15:36:39,11:41:24,03:20:01,14:23:26,03:32:34,08:23:01,11:45:35,13:51:09,03:28:25,09:19:42,13:02:38,10:48:18,04:08:06,09:57:35,09:26:42,17:00:12,12:19:10,11:03:30,17:00:36,18:38:52,15:15:37,13:37:52,12:59:33,20:01:48,17:45:20,14:57:22,12:05:19,18:23:11,18:59:58,05:23:35,07:37:20,19:35:06,04:32:25,15:49:32,13:57:33,23:24:14
5,mean_activity_end,15:28:26,13:34:56,15:55:29,11:44:21,19:10:19,12:27:42,07:29:29,14:16:07,06:45:41,10:58:58,12:16:08,17:08:10,08:47:18,12:15:39,13:39:53,13:30:32,08:07:37,12:26:01,12:03:29,16:22:56,13:49:59,11:24:36,16:49:18,17:39:29,15:16:23,13:12:36,16:16:13,19:28:37,15:04:11,15:59:49,12:36:13,16:30:39,18:46:11,11:50:24,10:59:36,18:41:13,06:11:20,17:54:20,15:48:24,23:40:02
6,median_activity_end,17:09:36,13:01:19,15:57:14,10:06:52,19:50:21,11:53:33,04:37:25,14:43:24,03:57:16,09:28:20,11:48:42,20:41:20,04:54:27,10:46:15,13:13:44,11:02:24,04:53:07,10:08:45,10:50:55,17:24:40,13:15:40,11:12:32,18:28:49,19:05:56,15:33:31,13:51:22,15:34:34,20:24:46,19:50:16,15:58:58,12:08:19,20:45:13,19:14:36,06:55:33,10:59:36,20:05:10,05:01:14,17:54:20,15:48:24,23:40:02
7,sum_activity_dhms,399:15:10:38,270:18:26:03,268:13:54:43,43:20:43:43,34:07:00:39,33:09:37:59,32:17:08:26,26:03:46:16,22:21:00:36,17:11:30:44,16:17:07:50,15:01:09:17,15:00:44:12,14:07:50:09,11:13:25:17,11:08:17:26,8:04:44:01,6:19:16:21,6:06:30:59,4:19:10:28,4:12:19:09,4:03:58:46,3:18:41:23,2:07:47:51,1:07:18:24,1:04:51:57,1:00:05:03,0:22:59:54,0:18:17:27,0:17:23:07,0:15:16:37,0:13:04:38,0:10:44:04,0:06:50:43,0:06:44:31,0:06:35:57,0:05:52:27,0:04:09:36,0:01:50:51,0:00:15:48
8,mean_activity_durations_hms,07:05:58,01:30:25,11:41:55,02:57:56,03:45:29,00:13:37,01:04:00,00:15:56,00:23:17,01:05:13,00:02:55,05:38:35,01:00:17,01:40:09,00:10:39,00:11:39,00:43:05,00:09:19,01:24:24,00:21:12,00:47:47,00:08:43,03:07:38,00:25:10,00:32:57,00:19:41,02:11:22,00:26:32,02:01:56,01:09:32,00:20:22,02:10:46,00:14:19,02:16:54,03:22:16,00:24:45,00:35:15,02:04:48,01:50:51,00:15:48
9,std_activity_durations_hms,02:53:40,01:19:51,01:47:24,02:06:40,02:08:35,00:09:06,00:42:11,00:12:46,00:16:23,00:06:53,00:03:00,03:59:45,00:39:46,01:08:52,00:10:11,00:07:58,00:28:13,00:05:14,00:41:30,00:22:59,00:24:27,00:06:30,02:46:36,00:15:47,00:42:55,00:25:24,00:29:51,00:32:21,00:39:10,00:13:44,00:20:32,00:29:51,00:01:48,00:49:41,00:07:01,00:05:41,00:16:10,01:56:05,00:00:00,00:00:00


In [19]:
activity_df_totals

,index,activity_df_totals
0,total_durations_seconds_in_df,110900640.0
1,total_durations_dhms_in_df,1283:13:44:00
2,total_days_in_df,1168
3,num_unique_activities_in_df,40
4,total_instances_of_activities_in_df,30162
5,total_activities_per_day_mean,26.94
6,total_activities_per_day_std,6.98
7,total_activities_per_day_min,1
8,total_activities_per_day_25,22
9,total_activities_per_day_median,27


In [66]:
ActivityStats(df=original_df).mean_std_durations()

Created 'mean_std_durations'


,Activity,mean_std_durations
0,Sleeping,07:05:58 ± 02:53:40
1,Internet,01:30:25 ± 01:19:51
2,Working,11:41:55 ± 01:47:24
3,Games,02:57:56 ± 02:06:40
4,Family Time,03:45:29 ± 02:08:35
5,Eating,00:13:37 ± 00:09:06
6,Programming,01:04:00 ± 00:42:11
7,Grooming,00:15:56 ± 00:12:46
8,Reading,00:23:17 ± 00:16:23
9,Gym,01:05:13 ± 00:06:53


In [68]:
mean_std_durations.to_csv('data/mean_std_durations.csv', index=None, encoding='utf-8')

In [20]:
def merged_df(df=df):
    # to get df's for heatmap, add to ActivityStats after getting it working
    globals()['merged_df_list'] = []
    for activity in unique_activities_sorted:
        globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
        globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Day']
        globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] =  df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']       
        globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
        globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]

        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').sum()).reset_index().rename({'Duration in Seconds':f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
        #print(f"{activity}:", globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].columns) # debugging when the names change
        globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').count()).reset_index().rename({'Duration in Seconds':f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')]
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]

        #print(type(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')])) # debugging when dataframe changes

        # add percentage of counts of activity / total counts activities in day as another column? maybe others to correlate as well

    #     end = pd.to_datetime(df['Day'].iloc[-1])
    #     start = pd.to_datetime(df['Day'][0])

        # https://www.kite.com/python/answers/how-to-insert-a-row-into-a-pandas-dataframe
        # https://stackoverflow.com/questions/19324453/add-missing-dates-to-pandas-dataframe
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.asfreq.html#pandas.DataFrame.asfreq
    #     row_df_start = pd.DataFrame({'Day':[start],
    #                           'Duration in Seconds' : 0,
    #                           #'Counts' : 0
    #                           })
    #     row_df_end = pd.DataFrame({'Day':[end],
    #                           'Duration in Seconds' : 0,
    #                           #'Counts' : 0
    #                           })

    #     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.concat([row_df_start, globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]], ignore_index=True)
    #     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.concat([globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_'), row_df_end]], ignore_index=True)

        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].set_index('Day')
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].asfreq('D', fill_value=0).reset_index()
    #     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].drop([0, 1]).reset_index()
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = 0
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')] = 0

        for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].index:
            globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')][i] = str(pd.Timedelta(seconds=globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')][i]))[-8:]
            globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')][i] = round(np.nan_to_num(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].rolling(7).mean()[i]))

        globals()['merged_df_list'].append(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')])
        
    globals()['activity_correlations_df'] = reduce(lambda  left, right: pd.merge(left, right, on=['Day'], how='outer'), globals()['merged_df_list']).fillna(0)
    
    # export 
    globals()['activity_correlations_df'].to_csv('data/activity_correlations_df.csv', index=None, encoding='utf-8')

    return globals()['activity_correlations_df']


        # there is a more efficient way for the above loop with .loc or .apply but idk how to get it to work
        # https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
        #grouped_programming_df.loc[grouped_programming_df['Seconds'], ('Duration in Seconds', i)] = str(pd.Timedelta(seconds=grouped_programming_df['Seconds']))[-8:]    

    #     del globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['index']

    # need either to trim it to single year, or leave be and make the trimming in another function/the heatmap builder
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.truncate.html
    # should look into pd.df.interpolate https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html

In [21]:
# if you ever return here, grouped_<activity>_df is what could have been used for the Github Heatmaps instead of whatever you did
merged_df(df=df)

/tmp/ipykernel_127605/3952998550.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')][i] = str(pd.Timedelta(seconds=globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')][i]))[-8:]
/tmp/ipykernel_127605/3952998550.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_secon

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,dat

In [22]:
def activity_correlations(activity_correlations_df=activity_correlations_df):
    activity_correlations_df.corr(method='pearson')
    
    globals()['rho'] = activity_correlations_df.corr().round(3)
    globals()['pval'] = activity_correlations_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    globals()['p'] = globals()['pval'].applymap(lambda x: ''.join(['*' for t in [0.001, 0.01, 0.05] if x<=t])) # can add more * for even smaller p value's e.g. 0.00001, 0.0001, in the list 
    globals()['correlation_matrix'] = globals()['rho'].astype(str) + p
    
    # export 
    globals()['rho'].reset_index().to_csv('data/rho.csv', index=None, encoding='utf-8')
    globals()['pval'].reset_index().to_csv('data/pval.csv', index=None, encoding='utf-8')
    globals()['correlation_matrix'].reset_index().to_csv('data/correlation_matrix.csv', index=None, encoding='utf-8')
    
    return globals()['correlation_matrix']

In [23]:
activity_correlations(activity_correlations_df=activity_correlations_df)

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_durations_seconds_7_day_rolling_mean,dancing_durations_seconds,dancing_counts,dancing_durations_seconds_7_day_rolling_mean
sleeping_durations_seconds,1.0***,0.374***,0.28***,0.015,-0.001,-0.036,-0.398***,-0.329***,-0.084**,-0.067*,-0.028,-0.057,-0.204***,-0.156***,-0.002,-0.27***,-0.303***,0.005,-0.193***,-0.212***,-0.058*,-0.085**,-0.132***,0.051,-0.223***,-0.252***,-0.012,-0.029,-0.038,0.014,-0.179***,-0.396***,0.003,-0.03,-0.024,0.018,-0.139***,-0.129***,0.034,0.032,0.047,0.027,-

# Timeline Charts

In [23]:
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

In [24]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.strftime.html
print(type(df['Start'][0]))

# df['Start Times'] = df['Start Times'].astype(str)
df['Start Times'] = df['Start'].dt.strftime('%H:%M:%S')
print(type(df['Start Times'][0]))
df

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


,Group,Activity,Start,End,Day,Duration in Seconds,Duration,Start Times
0,Computer,Internet,1970-01-01 23:24:10,1970-01-01 23:59:59,2019-12-30,2149.0,00:35:49,23:24:10
31524,Computer,Internet,1970-01-01 00:00:01,1970-01-01 01:06:34,2019-12-31,3993.0,01:06:33,00:00:01
31523,Personal,Personal,1970-01-01 01:06:36,1970-01-01 01:12:55,2019-12-31,379.0,00:06:19,01:06:36
31522,Computer,Internet,1970-01-01 01:13:03,1970-01-01 01:21:04,2019-12-31,481.0,00:08:01,01:13:03
31521,NaN,Grooming,1970-01-01 01:21:40,1970-01-01 01:27:22,2019-12-31,342.0,00:05:42,01:21:40
...,...,...,...,...,...,...,...,...
1365,NaN,Eating,1970-01-01 23:14:19,1970-01-01 23:25:45,2023-03-12,686.0,00:11:26,23:14:19
1364,NaN,Studying,1970-01-01 23:35:58,1970-01-01 23:55:19,2023-03-12,1161.0,00:19:21,23:35:58
1361,NaN,Toilet,1970-01-01 23:56:28,1970-01-01 23:59:59,2023-03-12,211.0,00:03:31,23:56:28
1370,NaN,Working,1970-01-01 00:00:01,1970-01-01 07:37:22,2023-03-13,27441.0,07:37:21,00:00:01


In [25]:
df['Day'][0]

datetime.date(2019, 12, 30)

In [26]:
df['Day'].iloc[-1]

datetime.date(2023, 3, 13)

In [84]:
customdata = ['Activity', 'Start Times', 'End', 'Duration']

In [864]:
# for overlapping bars, i.e. bullet chart
# https://stackoverflow.com/questions/67129085/how-to-get-plotly-express-timeline-overlapped-bars-to-be-visible

# how to fix varying thickness of bars
# https://community.plotly.com/t/bars-in-timeline-chart-from-plotly-express-equivalent-of-gantt-differs-in-width/44221
# https://stackoverflow.com/questions/64665728/plotly-how-to-prevent-varying-thickness-of-bars-in-a-gantt-diagram
# https://stackoverflow.com/questions/67129085/how-to-get-plotly-express-timeline-overlapped-bars-to-be-visible
# https://github.com/plotly/plotly.py/issues/2947

# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/66905297/plotly-is-it-possible-to-add-a-second-y-axis-to-plotly-timeline

# dropdown
# https://community.plotly.com/t/advanced-dropdown-menus-with-plotly/40922
# https://www.kaggle.com/benhamner/python-plotly-dropdown-demo
# https://www.kaggle.com/jrmistry/plotly-how-to-change-plot-data-using-dropdowns
# https://www.geeksforgeeks.org/how-to-make-dropdown-menus-in-plotly/
# https://stackoverflow.com/questions/59406167/plotly-how-to-filter-a-pandas-dataframe-using-a-dropdown-menu


def make_activity_dashboard(df=df, series=None):

    if series == None:
        title = "Activity Timeline Dashboard"
    else:
        title = f'{series} Timeline Dashboard'
    
    fig = go.Figure()
    fig = px.timeline(df, 
                      x_start="Start", 
                      x_end="End", 
                      y="Day", 
                      color='Activity', 
                      # https://stackoverflow.com/questions/59713016/plotly-express-how-to-fix-the-color-mapping-when-setting-color-by-column-name
                      color_discrete_map=colors,
                      category_orders={'Activity': [i for i in unique_activities_sorted]},
                      custom_data = customdata,
                      #template="plotly_dark",
                      width=1600,
                      height=1600
                     )
        
    fig.update_traces(hovertemplate =
                      #"Group: %{customdata[0]}<br>" +
                      "Activity: %{customdata[0]}<br>" +
                      "Day: %{y}<br>" +
                      "Start: %{customdata[1]}<br>" +
                      "End: %{x}<br>" +
                      "Duration: %{customdata[3]}" +
                      "<extra></extra>",
                      #width=10,
                     )
    
    # https://plotly.com/python/marker-style/
    #fig.add_trace(go.Scatter(x=df['Start'], y=df['Day'] , name='Starts', mode='markers', marker_color='#00CC96'))
    #fig.add_trace(go.Scatter(x=df['End'], y=df['Day'] , name='Ends', mode='markers', marker_color='#EF553B'))
    
    #fig.add_vline(x=median_sleep_start, line_dash="dash")
    #fig.add_vline(x=median_sleep_end, line_dash="dash")
    
    # https://github.com/d3/d3-time-format#locale_format change time format
    fig.update_layout(xaxis=dict(title='Timestamp',
                                 tickformat = '%H:%M:%S',
                                 range=['1970-01-01 00:00:00','1970-01-01 23:59:59'],
                                 fixedrange=True, # for keeping zoom level to entire days
                                ),
                      yaxis=dict(title='Day',
                                 tickformat = '%B, %d, %Y',
                                 dtick="M1",
                                 type="category",
                                 tickmode = "auto",
                                 #ticktext=np.arange(df['Day'][0], df['Day'].iloc[-1], 7)
                                 #ticktext=df['Day'], 
                                 #tickvals=df['Day'],
                                 #range=[]
                       ),
                      #hovermode='x unified',
                      #dragmode='pan',
                      title=dict(text=title, 
                                 yanchor="top",
                                 y=.98,
                                 xanchor="left",
                                 x=.05),
                      legend=dict(itemclick='toggle',
                                  itemdoubleclick='toggleothers',
                                  
    # # for putting legend below chart 
    #                               orientation="h",
    #                               yanchor="bottom",
    #                               y=-.1,
    #                               xanchor="right",
    #                               x=1,
                      )
                     )
    
    # https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
    fig.update_layout(plot_bgcolor='#282F44',
                      paper_bgcolor ='#282F44',
                      font_color='#FFF',
                      #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                      #hovermode='x unified',
                     )
    
    fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
    
    app = jupyter_dash.JupyterDash(__name__,
                                   external_stylesheets=external_stylesheets,
                                   title=f"{title}")
    
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
    #         style={'height': '98vh',
    #                'width': '98vw',
    #               },
            
    #         style={'height': '2500px',
    #                'width': '3250px',
    #               },
        ),
    ])

    title = title.replace('/', '_')
    fig.write_image(f"data/images/timeline/{title}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8050, debug=False)

In [138]:
make_activity_dashboard(df)

Dash app running on http://127.0.0.1:8050/


In [133]:
df['Activity'].unique()

array(['Internet', 'Personal', 'Grooming', 'Toilet', 'Sleeping',
       'Cooking', 'Eating', 'Games', 'Cleaning', 'Writing', 'Movies/TV',
       'Walking', 'Working', 'Reading', 'Driving',
       'Errands and Appointments', 'Family Time', 'Shopping', 'Phone',
       'Data Analysis', 'Dishes', 'Drumming', 'Programming',
       'Friends Time', 'Drinking', 'Dating', 'Gym', 'Keyboard', 'Drawing',
       'Swimming/Tanning', 'Concert/Event', 'Dancing', 'Studying',
       'Class', 'Biking', 'GameDev', 'Mycology', 'Club/Social Group',
       'Journaling', 'Traveling '], dtype=object)

In [143]:
# make_activity_dashboard(df.loc[df['Activity'] == 'Sleeping'], 'Sleeping')

Dash app running on http://127.0.0.1:8050/


In [147]:
for activity in df['Activity'].unique():
    make_activity_dashboard(df.loc[df['Activity'] == activity], activity)

Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app running on http://127.0.0.1:8050/
Dash app ru

In [148]:
activity_df_transposed

Activity,index,Sleeping,Internet,Working,Games,Family Time,Eating,Programming,Grooming,Reading,Gym,Toilet,Friends Time,Studying,Movies/TV,Driving,Walking,Writing,Cooking,Class,Cleaning,Dishes,Personal,Dating,Shopping,Mycology,Errands and Appointments,Club/Social Group,Drinking,GameDev,Swimming/Tanning,Phone,Keyboard,Journaling,Drawing,Traveling,Drumming,Data Analysis,Concert/Event,Biking,Dancing
0,total_num_instance_activity,1351,4312,551,355,219,3533,736,2364,1415,386,8238,64,359,206,1564,1402,274,1051,107,326,136,688,29,133,57,88,11,52,9,15,45,6,45,3,2,16,10,2,1,1
1,mean_num_activity_per_day,1.16,3.69,0.47,0.3,0.19,3.02,0.63,2.02,1.21,0.33,7.05,0.05,0.31,0.18,1.34,1.2,0.23,0.9,0.09,0.28,0.12,0.59,0.02,0.11,0.05,0.08,0.01,0.04,0.01,0.01,0.04,0.01,0.04,0.0,0.0,0.01,0.01,0.0,0.0,0.0
2,activity_percent,34.215101,23.1822,22.994834,3.755453,2.935969,2.859707,2.800872,2.23948,1.958512,1.496548,1.430974,1.288366,1.286875,1.226583,0.98966,0.971356,0.701818,0.58245,0.536945,0.410868,0.386413,0.356662,0.323522,0.199049,0.111682,0.102975,0.085917,0.082043,0.06525,0.062019,0.054498,0.046651,0.038294,0.02442,0.024051,0.023542,0.020955,0.01484,0.006591,0.000939
3,mean_activity_start,09:33:52,12:10:07,09:58:04,09:06:27,15:24:50,12:14:13,06:29:55,14:00:12,06:22:43,09:53:45,12:13:13,12:23:03,07:50:48,10:45:58,13:29:15,13:18:55,07:26:03,12:16:42,10:39:05,16:01:48,13:02:11,11:15:55,14:09:16,17:14:18,14:43:26,12:52:55,14:04:51,19:03:04,13:14:27,14:50:17,12:16:43,14:38:34,18:31:52,09:33:29,07:37:20,18:16:28,05:36:05,15:49:32,13:57:33,23:24:14
4,median_activity_start,10:41:17,11:31:13,07:56:00,06:45:48,15:36:39,11:41:24,03:20:01,14:23:26,03:32:34,08:23:01,11:45:35,13:51:09,03:28:25,09:19:42,13:02:38,10:48:18,04:08:06,09:57:35,09:26:42,17:00:12,12:19:10,11:03:30,17:00:36,18:38:52,15:15:37,13:37:52,12:59:33,20:01:48,17:45:20,14:57:22,12:05:19,18:23:11,18:59:58,05:23:35,07:37:20,19:35:06,04:32:25,15:49:32,13:57:33,23:24:14
5,mean_activity_end,15:28:26,13:34:56,15:55:29,11:44:21,19:10:19,12:27:42,07:29:29,14:16:07,06:45:41,10:58:58,12:16:08,17:08:10,08:47:18,12:15:39,13:39:53,13:30:32,08:07:37,12:26:01,12:03:29,16:22:56,13:49:59,11:24:36,16:49:18,17:39:29,15:16:23,13:12:36,16:16:13,19:28:37,15:04:11,15:59:49,12:36:13,16:30:39,18:46:11,11:50:24,10:59:36,18:41:13,06:11:20,17:54:20,15:48:24,23:40:02
6,median_activity_end,17:09:36,13:01:19,15:57:14,10:06:52,19:50:21,11:53:33,04:37:25,14:43:24,03:57:16,09:28:20,11:48:42,20:41:20,04:54:27,10:46:15,13:13:44,11:02:24,04:53:07,10:08:45,10:50:55,17:24:40,13:15:40,11:12:32,18:28:49,19:05:56,15:33:31,13:51:22,15:34:34,20:24:46,19:50:16,15:58:58,12:08:19,20:45:13,19:14:36,06:55:33,10:59:36,20:05:10,05:01:14,17:54:20,15:48:24,23:40:02
7,sum_activity_dhms,399:15:10:38,270:18:26:03,268:13:54:43,43:20:43:43,34:07:00:39,33:09:37:59,32:17:08:26,26:03:46:16,22:21:00:36,17:11:30:44,16:17:07:50,15:01:09:17,15:00:44:12,14:07:50:09,11:13:25:17,11:08:17:26,8:04:44:01,6:19:16:21,6:06:30:59,4:19:10:28,4:12:19:09,4:03:58:46,3:18:41:23,2:07:47:51,1:07:18:24,1:04:51:57,1:00:05:03,0:22:59:54,0:18:17:27,0:17:23:07,0:15:16:37,0:13:04:38,0:10:44:04,0:06:50:43,0:06:44:31,0:06:35:57,0:05:52:27,0:04:09:36,0:01:50:51,0:00:15:48
8,mean_activity_durations_hms,07:05:58,01:30:25,11:41:55,02:57:56,03:45:29,00:13:37,01:04:00,00:15:56,00:23:17,01:05:13,00:02:55,05:38:35,01:00:17,01:40:09,00:10:39,00:11:39,00:43:05,00:09:19,01:24:24,00:21:12,00:47:47,00:08:43,03:07:38,00:25:10,00:32:57,00:19:41,02:11:22,00:26:32,02:01:56,01:09:32,00:20:22,02:10:46,00:14:19,02:16:54,03:22:16,00:24:45,00:35:15,02:04:48,01:50:51,00:15:48
9,std_activity_durations_hms,02:53:40,01:19:51,01:47:24,02:06:40,02:08:35,00:09:06,00:42:11,00:12:46,00:16:23,00:06:53,00:03:00,03:59:45,00:39:46,01:08:52,00:10:11,00:07:58,00:28:13,00:05:14,00:41:30,00:22:59,00:24:27,00:06:30,02:46:36,00:15:47,00:42:55,00:25:24,00:29:51,00:32:21,00:39:10,00:13:44,00:20:32,00:29:51,00:01:48,00:49:41,00:07:01,00:05:41,00:16:10,01:56:05,00:00:00,00:00:00


In [151]:
# activity_df_transposed.to_csv('data/activity_df_transposed.csv', index=None, encoding='utf-8')

In [149]:
activity_df_totals

,index,activity_df_totals
0,total_durations_seconds_in_df,110900640.0
1,total_durations_dhms_in_df,1283:13:44:00
2,total_days_in_df,1168
3,num_unique_activities_in_df,40
4,total_instances_of_activities_in_df,30162
5,total_activities_per_day_mean,26.94
6,total_activities_per_day_std,6.98
7,total_activities_per_day_min,1
8,total_activities_per_day_25,22
9,total_activities_per_day_median,27


In [152]:
# activity_df_totals.to_csv('data/activity_df_totals.csv', index=None, encoding='utf-8')

In [24]:
df_counts = [i for i in activity_correlations_df.columns if i.endswith('counts')]
df_durations_seconds = [i for i in activity_correlations_df.columns if i.endswith('seconds')]
df_durations = [i for i in activity_correlations_df.columns if i.endswith('durations')]
df_durations_seconds_7_day_rolling_mean = [i for i in activity_correlations_df.columns if i.endswith('mean')]

# Create correlation_colors, must create colors from above and match activity names

In [25]:
# how to make a new color dictionary based on the original dictionary, 
# could make another one for other df's.columns e.g. <current> line 5 of this cell
correlation_colors = {}
for k, v in colors.items():
#     print(k)
    for col in activity_correlations_df.columns:
        if k.lower().replace('/', '_').replace(' ', '_') in col:
#             print(col)
            correlation_colors[col] = v
correlation_colors

{'working_durations_seconds': '#D50000',
 'working_counts': '#D50000',
 'working_durations': '#D50000',
 'working_durations_seconds_7_day_rolling_mean': '#D50000',
 'sleeping_durations_seconds': '#2196F3',
 'sleeping_counts': '#2196F3',
 'sleeping_durations': '#2196F3',
 'sleeping_durations_seconds_7_day_rolling_mean': '#2196F3',
 'grooming_durations_seconds': '#80D8FF',
 'grooming_counts': '#80D8FF',
 'grooming_durations': '#80D8FF',
 'grooming_durations_seconds_7_day_rolling_mean': '#80D8FF',
 'internet_durations_seconds': '#00C853',
 'internet_counts': '#00C853',
 'internet_durations': '#00C853',
 'internet_durations_seconds_7_day_rolling_mean': '#00C853',
 'movies_tv_durations_seconds': '#00C853',
 'movies_tv_counts': '#00C853',
 'movies_tv_durations': '#00C853',
 'movies_tv_durations_seconds_7_day_rolling_mean': '#00C853',
 'programming_durations_seconds': '#00E676',
 'programming_counts': '#00E676',
 'programming_durations': '#00E676',
 'programming_durations_seconds_7_day_rollin

In [2]:
activity_correlations_df = pd.read_csv('data/activity_correlations_df.csv')
activity_correlations_df

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,dat

# Bar Charts

In [843]:
def make_activity_stacked_barcharts(df=activity_correlations_df, 
                                    series=None, 
                                    df_durations_subset=df_durations_subset, 
                                    df_durations_seconds_subset=df_durations_seconds_subset,
                                    df_durations=df_durations,
                                    df_counts=df_counts,
                                    df_counts_subset=df_counts_subset,
                                    df_durations_seconds=df_durations_seconds,
                                    correlation_colors=correlation_colors
                                    ):

    rolling = 7

    if series == None:
        title = 'Activity Timeline Stacked Barchart'
        customdata=[df[col] for col in df[df_durations].columns],
        fig = px.bar(df, 
                 x=df['Day'], 
                 y=[df[col] for col in df[df_durations_seconds].columns],
                 color_discrete_map=correlation_colors,
                 title=title,
                 width=2500,
                 height=1000
                )
        fig.update_traces(customdata=customdata,
                      hovertemplate =
    #                   "Activity: {}<br>" +
                      "Day: %{x}<br>" +
                      "Duration in Seconds: %{y}<br>" +
    #                   "Duration: %{customdata}<br>" +
                      "<extra></extra>",
                      #width=10,
                     )
        
    else:
        title = f'{series} Timeline Stacked Barchart'
        customdata=[col for col in df[df_durations_subset]],
        fig = px.bar(df, 
                 x=df['Day'], 
                 y=[col for col in df[df_durations_seconds_subset]],
                 color_discrete_map=correlation_colors,
                 color=df[df_durations_seconds_subset].name,
                 title=title,
                 width=2500,
                 height=1000,
                )

        fig.update_traces(customdata=customdata,
                          hovertemplate =
        #                   "Activity: {}<br>" +
                          "Day: %{x}<br>" +
                          "Duration in Seconds: %{y}<br>" +
        #                   "Duration: %{customdata}<br>" +
                          "<extra></extra>",
                          #width=10,
                          name=series,
                          marker=dict(color=correlation_colors[df[df_durations_seconds_subset].name])
                         )
    
    
    #fig = go.Figure()
    # for col in df[df_durations].columns:
    #     fig.add_trace(go.Bar(x=df["Day"], 
    #                          y=df[col],
    #                          name=df[col].name,
    # #                         marker_color=df[col], 
    #                          marker=dict(color_discrete_map=correlation_colors,)
    #                         ))
    
    # https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
    fig.update_layout(barmode='stack',
                      plot_bgcolor='#282F44',
                      paper_bgcolor ='#282F44',
                      font_color='#FFF',
                      #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                      #hovermode='x unified',
                      
                      # https://github.com/d3/d3-time-format#locale_format change time format
                      xaxis=dict(title='Day',
                                ),
                      yaxis=dict(title='Activities',
                                 fixedrange=True, # for keeping zoom level to entire days
                                ),
                      #hovermode='x unified',
                      #dragmode='pan',
                      title=dict(text=title,
                                 yanchor="top",
                                 y=.98,
                                 xanchor="left",
                                 x=.05),
                      legend=dict(#toggle=True,
                                  title='Activities',
                                  itemclick='toggle',
                                  itemdoubleclick='toggleothers',
                                  # for putting legend below chart 
    #                               orientation="h",
    #                               yanchor="bottom",
    #                               y=-.1,
    #                               xanchor="right",
    #                               x=1,
                                 ),
                     )
    fig['data'][0]['showlegend']=True

    app = jupyter_dash.JupyterDash(__name__,
                                   external_stylesheets=external_stylesheets,
                                   title=f"{title}")
                  
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
            style={'height': '98vh',
                   'width': '98vw',
                  },
            
    #         style={'height': '950px',
    #                'width': '1600px',
    #               },
        ),
    ])

    title = title.replace('/', '_')
    fig.write_image(f"data/images/barcharts/{title}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8080, debug=False)

In [844]:
make_activity_stacked_barcharts(df=activity_correlations_df)

Dash app running on http://127.0.0.1:8080/


In [293]:
# [col for col in activity_correlations_df if col.startswith('sleeping')]

In [29]:
# activities = []
# for activity in [col for col in activity_correlations_df if col.startswith('sleeping')]:
    # activities.append(activity)
# activities

In [26]:
activities_combined = []
for activity in unique_activities_sorted:
    activities = []
    for a in [col for col in activity_correlations_df if col.startswith(activity.lower().replace('/', '_').replace(' ', '_'))]:
        activities.append(a)
    
    activities.append('Day')
    activities_combined.append(activities)
activities_combined

[['sleeping_durations_seconds',
  'sleeping_counts',
  'sleeping_durations',
  'sleeping_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['internet_durations_seconds',
  'internet_counts',
  'internet_durations',
  'internet_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['working_durations_seconds',
  'working_counts',
  'working_durations',
  'working_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['games_durations_seconds',
  'games_counts',
  'games_durations',
  'games_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['family_time_durations_seconds',
  'family_time_counts',
  'family_time_durations',
  'family_time_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['eating_durations_seconds',
  'eating_counts',
  'eating_durations',
  'eating_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['programming_durations_seconds',
  'programming_counts',
  'programming_durations',
  'programming_durations_seconds_7_day_rolling_mean',
  'Day'],
 ['grooming_durations_seconds',
  

In [861]:
def make_all_barcharts(activity_correlations_df=activity_correlations_df, 
                       df_durations_seconds=df_durations_seconds,
                       df_durations=df_durations,
                       df_counts=df_counts,
                       df_counts_subset=df_counts_subset,
                       activities_combined=activities_combined,
                       unique_activities_sorted=unique_activities_sorted,
                       correlation_colors=correlation_colors):

    for df, activity in zip(activities_combined, unique_activities_sorted):
        activity_correlations_df_subset = activity_correlations_df[df]
        df_durations_seconds_subset = df[0]
        df_durations_subset = df[2]

        #return activity_correlations_df
        make_activity_stacked_barcharts(df=activity_correlations_df_subset, df_durations_subset=df_durations_subset, df_durations_seconds_subset=df_durations_seconds_subset, df_counts=df_counts, df_counts_subset=df_counts_subset, correlation_colors=correlation_colors, series=activity)

In [862]:
make_all_barcharts()

Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app running on http://127.0.0.1:8080/
Dash app ru

# Scatterplots

In [1022]:
def make_scatter_dashboard(df=activity_correlations_df, 
                            series=None, 
                            df_durations_subset=df_durations_subset, 
                            df_durations_seconds_subset=df_durations_seconds_subset,
                            df_durations=df_durations,
                            df_counts=df_counts,
                            df_counts_subset=df_counts_subset,
                            df_durations_seconds=df_durations_seconds,
                            correlation_colors=correlation_colors):

    activity_correlations_df['Day'] = pd.to_datetime(activity_correlations_df['Day'])

    rolling = 7

    if series == None:
        title = 'Activity Timeline Scatter Plot'
        customdata=[df[col] for col in df[df_durations].columns],
        fig = px.scatter(df, 
                     x=df['Day'], 
                     y=[df[col] for col in df[df_durations_seconds].columns],
                     color_discrete_map=correlation_colors,
                     title=title,
                     # https://plotly.com/python/linear-fits/
                     trendline="ols",
    #                  trendline="rolling",
    #                  trendline_options=dict(window=rolling),
                     width=1600,
                     height=1600
                    )
    
        fig.update_traces(customdata=customdata,
                          hovertemplate =
        #                   "Activity: {}<br>" +
                          "Day: %{x}<br>" +
                          "Duration in Seconds: %{y}<br>" +
        #                   "Duration: %{customdata}<br>" +
                          "<extra></extra>",
                          #width=10,
                         )
    
    else:
        title = f'{series} Timeline Scatter Plot'
        customdata=[col for col in df[df_durations_subset]],
        fig = px.scatter(df, 
                 x=df['Day'], 
                 y=[col for col in df[df_durations_seconds_subset]],
                 color_discrete_map=correlation_colors,
                 color=df[df_durations_seconds_subset].name,
                 title=title,
                 # https://plotly.com/python/linear-fits/
                 trendline="ols",
    #                  trendline="rolling",
    #                  trendline_options=dict(window=rolling),
                 width=1600,
                 height=1600,
                )

        fig.update_traces(customdata=customdata,
                          hovertemplate =
        #                   "Activity: {}<br>" +
                          "Day: %{x}<br>" +
                          "Duration in Seconds: %{y}<br>" +
        #                   "Duration: %{customdata}<br>" +
                          "<extra></extra>",
                          #width=10,
                          name=series,
                          marker=dict(color=correlation_colors[df[df_durations_seconds_subset].name])
                         )
    
    
    #fig = go.Figure()
    # for col in df[df_durations].columns:
    #     fig.add_trace(go.Bar(x=df["Day"], 
    #                          y=df[col],
    #                          name=df[col].name,
    # #                         marker_color=df[col], 
    #                          marker=dict(color_discrete_map=correlation_colors,)
    #                         ))
    
    # https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
    fig.update_layout(barmode='stack',
                      plot_bgcolor='#282F44',
                      paper_bgcolor ='#282F44',
                      font_color='#FFF',
                      #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                      #hovermode='x unified',
                      
                      # https://github.com/d3/d3-time-format#locale_format change time format
                      xaxis=dict(title='Day',
                                ),
                      yaxis=dict(title='Durations in Seconds',
    #                              fixedrange=True, # for keeping zoom level to entire days
                                ),
                      #hovermode='x unified',
                      #dragmode='pan',
                      title=dict(text=title, 
                                 yanchor="top",
                                 y=.98,
                                 xanchor="left",
                                 x=.05),
                      legend=dict(title='Activities',
                                  itemclick='toggle',
                                  itemdoubleclick='toggleothers',
                                  # for putting legend below chart 
    #                               orientation="h",
    #                               yanchor="bottom",
    #                               y=-.1,
    #                               xanchor="right",
    #                               x=1,
                                 ),
                     )

    fig['data'][0]['showlegend']=True
    
    app = jupyter_dash.JupyterDash(__name__,
                                   external_stylesheets=external_stylesheets,
                                   title=f"{title}")
                  
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
            style={'height': '98vh',
                   'width': '98vw',
                  },
            
    #         style={'height': '950px',
    #                'width': '1600px',
    #               },
        ),
    ])

    title = title.replace('/', '_')
    fig.write_image(f"data/images/scatterplots/{title}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8095, debug=False)

In [1023]:
make_scatter_dashboard(df=activity_correlations_df)

Dash app running on http://127.0.0.1:8095/


In [51]:
def make_all_scatterplots(activity_correlations_df=activity_correlations_df, 
                       df_durations_seconds=df_durations_seconds,
                       df_durations=df_durations,
                       df_counts=df_counts,
                       df_counts_subset=df_counts_subset,
                       activities_combined=activities_combined,
                       unique_activities_sorted=unique_activities_sorted,
                       correlation_colors=correlation_colors):

    for df, activity in zip(activities_combined, unique_activities_sorted):
        activity_correlations_df_subset = activity_correlations_df[df]
        df_durations_seconds_subset = df[0]
        df_durations_subset = df[2]

        #return activity_correlations_df
        make_scatter_dashboard(df=activity_correlations_df_subset, df_durations_subset=df_durations_subset, df_durations_seconds_subset=df_durations_seconds_subset, df_counts=df_counts,df_counts_subset=df_counts_subset, correlation_colors=correlation_colors, series=activity)

In [52]:
make_all_scatterplots()

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,dat

# Histograms

In [1016]:
def make_histogram_dashboard(df=activity_correlations_df, 
                            series=None, 
                            df_durations_subset=df_durations_subset,
                            df_durations_seconds_subset=df_durations_seconds_subset,
                            df_durations=df_durations,
                            df_durations_seconds=df_durations_seconds,
                            df_counts=df_counts,
                            df_counts_subset=df_counts_subset,
                            correlation_colors=correlation_colors):

    if series == None:
        title = 'Activity Timeline Histogram'
        customdata=[df[col] for col in df[df_durations].columns],
        fig = px.histogram(df, 
                       #x=df['Day'], 
                       x=[df[col] for col in df[df_durations_seconds].columns],
                       color_discrete_map=correlation_colors,
                       title=title,
                       width=1600,
                       height=1600)
    
        fig.update_traces(customdata=customdata,
                      hovertemplate =
    #                   "Activity: {}<br>" +
                      "Counts: %{y}<br>" +
                      "Duration in Seconds: %{x}<br>" +
    #                   "Duration: %{customdata}<br>" +
                      "<extra></extra>",
                      #width=10,
                     )
    else:
        title = f'{series} Timeline Histogram'
        customdata=[col for col in df[df_durations_subset]],
        fig = px.histogram(df, 
                 #x=df['Day'], 
                 x=[col for col in df[df_durations_seconds_subset]],
                 color_discrete_map=correlation_colors,
                 color=df[df_durations_seconds_subset].name,
                 title=title,
                 width=1600,
                 height=1600
                )

        fig.update_traces(customdata=customdata,
                          hovertemplate =
        #                   "Activity: {}<br>" +
                          "Day: %{x}<br>" +
                          "Duration in Seconds: %{y}<br>" +
        #                   "Duration: %{customdata}<br>" +
                          "<extra></extra>",
                          #width=10,
                          name=series,
                          marker=dict(color=correlation_colors[df[df_durations_seconds_subset].name]),
                          showlegend=False,
                         )
    
    #fig = go.Figure()
    # for col in df[df_durations].columns:
    #     fig.add_trace(go.Bar(x=df["Day"], 
    #                          y=df[col],
    #                          name=df[col].name,
    # #                         marker_color=df[col], 
    #                          marker=dict(color_discrete_map=correlation_colors,)
    #                         ))
    
    # https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
    fig.update_layout(barmode='stack',
                      bargap=0.1,
                      plot_bgcolor='#282F44',
                      paper_bgcolor ='#282F44',
                      font_color='#FFF',
                      #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                      #hovermode='x unified',
                      
                      # https://github.com/d3/d3-time-format#locale_format change time format
                      xaxis=dict(title='Durations in Seconds',
                                 #type='date',
                                 tickformat="%H:%M:%S",
                                ),
                      yaxis=dict(title='Count',
    #                              fixedrange=True, # for keeping zoom level to entire days
                                ),
                      #hovermode='x unified',
                      #dragmode='pan',
                      title=dict(text=title, 
                                 yanchor="top",
                                 y=.98,
                                 xanchor="left",
                                 x=.05),
                      showlegend=True,
                      legend=dict(title='Activities',
                                  itemclick='toggle',
                                  itemdoubleclick='toggleothers',
                                  # for putting legend below chart 
    #                               orientation="h",
    #                               yanchor="bottom",
    #                               y=-.1,
    #                               xanchor="right",
    #                               x=1,
                                 ),
                     )
    fig['data'][0]['showlegend']=True
    
    app = jupyter_dash.JupyterDash(__name__,
                                   external_stylesheets=external_stylesheets,
                                   title=f"{title}")
                  
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
            style={'height': '98vh',
                   'width': '98vw',
                  },
            
    #         style={'height': '950px',
    #                'width': '1600px',
    #               },
        ),
    ])

    title = title.replace('/', '_')
    fig.write_image(f"data/images/histograms/{title}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8110, debug=False)

In [338]:
make_histogram_dashboard(df=activity_correlations_df)

KeyError: 'Days'

In [1020]:
def make_all_histograms(activity_correlations_df=activity_correlations_df, 
                       df_durations_seconds=df_durations_seconds,
                       df_durations=df_durations, 
                       df_counts=df_counts,
                       df_counts_subset=df_counts_subset,
                       activities_combined=activities_combined,
                       unique_activities_sorted=unique_activities_sorted,
                       correlation_colors=correlation_colors):

    for df, activity in zip(activities_combined, unique_activities_sorted):
        activity_correlations_df_subset = activity_correlations_df[df]
        df_durations_seconds_subset = df[0]
        df_durations_subset = df[2]

        #return activity_correlations_df
        make_histogram_dashboard(df=activity_correlations_df_subset, df_durations_subset=df_durations_subset, df_durations_seconds_subset=df_durations_seconds_subset, df_counts=df_counts, df_counts_subset=df_counts_subset, correlation_colors=correlation_colors, series=activity)

In [1021]:
make_all_histograms()

Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app running on http://127.0.0.1:8110/
Dash app ru

# Github Heatmaps

In [1026]:
activity_correlations_df

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,dat

In [164]:
years = list(activity_correlations_df['Day'].dt.year.unique())
years = [int(i) for i in years]
years

[2019, 2020, 2021, 2022, 2023]

In [160]:
tuple(years)

(2019, 2020, 2021, 2022, 2023)

In [55]:
# https://python.plainenglish.io/interactive-calendar-heatmaps-with-plotly-the-easieast-way-youll-find-5fc322125db7
# https://community.plotly.com/t/colored-calendar-heatmap-in-dash/10907/21
# https://github.com/brunorosilva/plotly-calplot


def make_github_heatmap_dashboard(df=activity_correlations_df, 
                            series=None, 
                            df_durations_subset=df_durations_subset, 
                            df_durations_seconds_subset=df_durations_seconds_subset,
                            df_durations=df_durations, 
                            df_durations_seconds=df_durations_seconds,
                            df_counts=df_counts,
                            df_counts_subset=df_counts_subset,
                            correlation_colors=correlation_colors, 
                            count=True, **kwargs):

    if count == True:
        title = f'{series} Counts Timeline Heatmap '
    
        fig = calplot(
                 df,
                 x='Day',
                 y=df_counts_subset,
                 dark_theme=True,
                 years_title=True,
                 showscale=True,
                 colorscale=[(0.00, "white"), (1.00, correlation_colors[df[df_durations_seconds_subset].name])],
                 title=title,
                 month_lines_width=3, 
                 month_lines_color="#444"
        )

        fig.update_layout(**kwargs)
        title = title.replace('/', '_')
        fig.write_image(f"data/images/github_heatmaps/counts/{title}.png")

    else:
        title = f'{series} Durations Timeline Heatmap'
    
        fig = calplot(
                 df,
                 x='Day',
                 y=df_durations_seconds_subset,
                 dark_theme=True,
                 years_title=True,
                 showscale=True,
                 colorscale=[(0.00, "white"), (1.00, correlation_colors[df[df_durations_seconds_subset].name])],
                 title=title,
                 month_lines_width=3, 
                 month_lines_color='#444'
        )

        fig.update_layout(**kwargs)
        title = title.replace('/', '_')
        fig.write_image(f"data/images/github_heatmaps/durations/{title}.png")

    app = jupyter_dash.JupyterDash(__name__,
                                    external_stylesheets=external_stylesheets,
                                    title=f"{title}")
                  
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
            style={'height': '98vh',
                   'width': '98vw',
                  },
            
    #         style={'height': '950px',
    #                'width': '1600px',
    #               },
        ),
    ])
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8110, debug=False)

In [330]:
# make_github_heatmap_dashboard(df=activity_correlations_df_subset)

In [28]:
for df, activity in zip(activities_combined[0:1], unique_activities_sorted[0:1]):
        activity_correlations_df_subset = activity_correlations_df[df]
        df_durations_seconds_subset = df[0]
        df_counts_subset = df[1]
        df_durations_subset = df[2]
activity_correlations_df_subset

,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,Day
0,41503.0,3.0,11:31:43,0.0,2019-12-31
1,26261.0,2.0,07:17:41,0.0,2020-01-01
2,55405.0,2.0,15:23:25,0.0,2020-01-02
3,41466.0,2.0,11:31:06,0.0,2020-01-03
4,15501.0,1.0,04:18:21,0.0,2020-01-04
...,...,...,...,...,...
1165,59984.0,1.0,16:39:44,32695.0,2023-03-10
1166,21918.0,1.0,06:05:18,32215.0,2023-03-11
1167,22703.0,1.0,06:18:23,32220.0,2023-03-12
1168,0.0,0.0,0,0.0,2019-12-30


In [247]:
# df_durations_seconds_subset

In [248]:
# df_durations_subset

In [287]:
# df_counts_subset

In [39]:
activity_correlations_df = activity_correlations_df.sort_values('Day')
activity_correlations_df = activity_correlations_df.set_index('Day')
activity_correlations_df = activity_correlations_df.reset_index()
activity_correlations_df

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,dat

In [61]:
def make_all_github_heatmaps(activity_correlations_df=activity_correlations_df, 
                       df_durations_seconds=df_durations_seconds,
                       df_durations=df_durations,
                       df_counts=df_counts,
                       activities_combined=activities_combined,
                       unique_activities_sorted=unique_activities_sorted,
                       correlation_colors=correlation_colors):

    activity_correlations_df = activity_correlations_df[2:]

    for df, activity in zip(activities_combined, unique_activities_sorted):
        activity_correlations_df_subset = activity_correlations_df[df]
        df_durations_seconds_subset = df[0]
        df_counts_subset = df[1]
        df_durations_subset = df[2]

        #return activity_correlations_df_subset
        make_github_heatmap_dashboard(df=activity_correlations_df_subset, 
                                        df_durations_subset=df_durations_subset, 
                                        df_durations_seconds_subset=df_durations_seconds_subset,
                                        df_counts=df_counts,
                                        df_counts_subset=df_counts_subset,
                                        correlation_colors=correlation_colors, 
                                        series=activity, 
                                        width=1600, height=1600, title=dict(yanchor="top",y=.99, xanchor="left", x=.05),
                                        count=True) # change to False if you want durations

In [62]:
make_all_github_heatmaps()

 * Running on http://127.0.0.1:8110 (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2023 08:38:17] "GET /_alive_ccb9ace8-f17b-478f-9e5c-19842dd7a162 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8110/


 * Running on http://127.0.0.1:8110 (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2023 08:38:19] "GET /_alive_ccb9ace8-f17b-478f-9e5c-19842dd7a162 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8110/


 * Running on http://127.0.0.1:8110 (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2023 08:38:21] "GET /_alive_ccb9ace8-f17b-478f-9e5c-19842dd7a162 HTTP/1.1" 200 -


...

Dash app running on http://127.0.0.1:8110/


 * Running on http://127.0.0.1:8110 (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2023 08:39:25] "GET /_alive_ccb9ace8-f17b-478f-9e5c-19842dd7a162 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8110/


 * Running on http://127.0.0.1:8110 (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2023 08:39:27] "GET /_alive_ccb9ace8-f17b-478f-9e5c-19842dd7a162 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8110/
